# 第十章 数据聚合与分组运算
- [GroupBy机制](#GroupBy机制)
    - groupby(..).size()
- [对分组进行迭代](#对分组进行迭代)
- [选取一列或列的子集](#选取一列或列的子集)

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 5
np.random.seed(12345)
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
%matplotlib inline

#### GroupBy机制

<img src='images/10_1.png'>

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [9]:
df.groupby('key1').mean()
df.groupby(['key1', 'key2']).mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

#### 对分组进行迭代

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023


In [14]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print (k1,k2)
    print (group)

a one
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
a two
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
b one
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
b two
  key1 key2    data1     data2
3    b  two -0.55573  0.769023


In [15]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


In [16]:
df.dtypes
grouped = df.groupby(df.dtypes, axis=1)

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [17]:
for dtype, group in grouped:
    print (dtype, group)

float64       data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
object   key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


#### 选取一列或列的子集

In [22]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [23]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

#### 通过字典或Series进行分组

In [18]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [19]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [20]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


In [21]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis=1).count()

a       red
b       red
      ...  
e       red
f    orange
Length: 6, dtype: object

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


#### 通过函数分组

In [24]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.591569,-0.993608,0.798764,-0.791374,2.119639
5,0.886429,-2.001637,-0.371843,1.669025,-0.438570
6,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [26]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.539741 -1.296221  0.274992 -1.021228 -0.577087
  two  0.124121  0.302614  0.523772  0.000940  1.343810
5 one  0.886429 -2.001637 -0.371843  1.669025 -0.438570
6 two -0.713544 -0.831154 -2.370232 -1.860761 -0.860757

#### 根据索引级别分组

In [27]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.560145 -1.265934  0.119827 -1.063512  0.332883
1     -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
2      0.286350  0.377984 -0.753887  0.331286  1.349742
3      0.069877  0.246674 -0.011862  1.004812  1.327195

In [28]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


#### 数据集合

<img src='images/10_2.png'>

In [29]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [30]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [31]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

#### 面向列的多函数应用

In [33]:
tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
...,...,...,...,...,...,...,...
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [34]:
grouped = tips.groupby(['day', 'smoker'])

In [35]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
                  ...   
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, Length: 8, dtype: float64

In [36]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
...               ...       ...           ...
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

[8 rows x 3 columns]

In [37]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
...               ...       ...
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

[8 rows x 2 columns]

In [38]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
...             ...       ...       ...        ...        ...    ...
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

[8 rows x 6 columns]

In [39]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
...            ...       ...       ...
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

[8 rows x 3 columns]

In [40]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
            Durchschnitt Abweichung Durchschnitt Abweichung
day  smoker                                                
Fri  No         0.151650   0.000791    18.420000  25.596333
     Yes        0.174783   0.002631    16.813333  82.562438
...                  ...        ...          ...        ...
Thur No         0.160298   0.001503    17.113111  59.625081
     Yes        0.163863   0.001551    19.190588  69.808518

[8 rows x 4 columns]

In [41]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip  size
day  smoker            
Fri  No      3.50     9
     Yes     4.73    31
...           ...   ...
Thur No      6.70   112
     Yes     5.00    40

[8 rows x 2 columns]

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
...               ...       ...       ...       ...  ...
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

[8 rows x 5 columns]

#### 以“没有索引”的形式返回聚合数据

In [43]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
...,...,...,...,...,...,...
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


#### apply 拆分 应用 合并

In [45]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
...,...,...,...,...,...,...,...
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [46]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
...                ...   ...    ...   ...     ...   ...       ...
Yes    178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

[10 rows x 7 columns]

In [47]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill   tip smoker   day    time  size   tip_pct
smoker day                                                            
No     Fri  94        22.75  3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33  9.00     No   Sat  Dinner     4  0.186220
...                     ...   ...    ...   ...     ...   ...       ...
Yes    Sun  182       45.35  3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11  5.00    Yes  Thur   Lunch     4  0.115982

[8 rows x 7 columns]

In [48]:
result = tips.groupby('smoker')['tip_pct'].describe()
result
result.unstack('smoker')

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


       smoker
count  No        151.000000
       Yes        93.000000
                    ...    
max    No          0.291990
       Yes         0.710345
Length: 16, dtype: float64

In [49]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill     tip  size   tip_pct
day  smoker                                          
Fri  No     count        4.00  4.0000  4.00  4.000000
            mean        18.42  2.8125  2.25  0.151650
...                       ...     ...   ...       ...
Thur Yes    75%         19.81  4.0000  2.00  0.194837
            max         43.11  5.0000  4.00  0.241255

[64 rows x 4 columns]

#### 禁止分组键

In [50]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
...,...,...,...,...,...,...,...
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


#### 分位桶和桶分析

In [51]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-1.23, 0.489]
1    (-2.956, -1.23]
          ...       
8     (0.489, 2.208]
9     (0.489, 2.208]
Name: data1, Length: 10, dtype: category
Categories (4, interval[float64]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [54]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.956, -1.23]",95.0,1.670835,-0.039521,-3.399312
"(-1.23, 0.489]",598.0,3.260383,-0.002051,-2.989741
"(0.489, 2.208]",297.0,2.954439,0.081822,-3.745356
"(2.208, 3.928]",10.0,1.765640,0.024750,-1.929776


In [55]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,1.670835,-0.049902,-3.399312
1,100.0,2.628441,0.030989,-1.950098
...,...,...,...,...
8,100.0,2.735527,0.103406,-3.745356
9,100.0,2.377020,0.220122,-2.064111


#### 示例：用特定于分组的值填充缺失值

In [58]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

0         NaN
1   -1.613716
       ...   
4         NaN
5   -0.614436
Length: 6, dtype: float64

0   -0.607214
1   -1.613716
       ...   
4   -0.607214
5   -0.614436
Length: 6, dtype: float64

In [59]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.691656
New York      0.758701
                ...   
California   -0.159134
Idaho        -0.417616
Length: 8, dtype: float64

In [60]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data
data.groupby(group_key).mean()

Ohio         -1.691656
New York      0.758701
                ...   
California   -0.159134
Idaho              NaN
Length: 8, dtype: float64

East   -0.657163
West    0.237989
dtype: float64

In [61]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -1.691656
New York      0.758701
                ...   
California   -0.159134
Idaho         0.237989
Length: 8, dtype: float64

In [65]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -1.691656
New York      0.758701
                ...   
California   -0.159134
Idaho        -1.000000
Length: 8, dtype: float64

#### 示例：随机采样和排列

In [66]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [67]:
deck[:13]

AH     1
2H     2
      ..
KH    10
QH    10
Length: 13, dtype: int64

In [68]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

9C     9
6S     6
4D     4
QS    10
4S     4
dtype: int64

In [69]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  7C    7
   3C    3
        ..
S  AS    1
   5S    5
Length: 8, dtype: int64

In [70]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C    5
6C    6
     ..
3S    3
7S    7
Length: 8, dtype: int64

#### 示例：分组加权平均数和相关系数

In [71]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,1.141487,0.041173
1,a,0.387803,0.167435
...,...,...,...
6,b,1.588909,0.104578
7,b,-0.064834,0.524368


In [72]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.217939
b    0.346551
dtype: float64

In [73]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [74]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [75]:
rets = close_px.pct_change().dropna()

In [76]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
...,...,...,...,...
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [77]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
          ...   
2010    0.571946
2011    0.581987
Length: 9, dtype: float64

#### 组级别的线性回归

In [78]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [79]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
...,...,...
2010,1.052608,0.001261
2011,0.806605,0.001514


#### 透视表和交叉表

In [80]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
...               ...       ...       ...         ...
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

[8 rows x 4 columns]

In [81]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
...               ...       ...       ...       ...
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

[6 rows x 4 columns]

In [82]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
...               ...       ...       ...       ...       ...       ...
Lunch  Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

[7 rows x 6 columns]

In [83]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [84]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                 Fri       Sat  Sun      Thur
time   size smoker                              
Dinner 1    No      0.0  0.137931  0.0  0.000000
            Yes     0.0  0.325733  0.0  0.000000
...                 ...       ...  ...       ...
Lunch  5    No      0.0  0.000000  0.0  0.121389
       6    No      0.0  0.000000  0.0  0.173706

[21 rows x 4 columns]

#### 交叉表crosstab

In [85]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [86]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
...,...,...,...
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [87]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [88]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
...          ...  ...  ...
Lunch  Thur   44   17   61
All          151   93  244

[7 rows x 3 columns]

In [89]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS